In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set a few plotting defaults
%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams['font.size'] = 18
plt.rcParams['patch.edgecolor'] = 'k'

In [ ]:
pd.options.display.max_columns = 150

#Read in data
train = pd.read_csv('../input/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('../input/costa-rican-household-poverty-prediction/test.csv')
train.head()

In [ ]:
train.info()

#130 개의 정수 열, 8 개의 부동 (숫자) 열 및 5 개의 개체 열이 있음
#정수 열은 부울 변수 (0 또는 1을 사용), 불연속 순서 값을 가진 서수 변수를 나타낸다.
#객체 열은 머신 러닝 모델로 직접 공급 될 수 없기 때문에 문제가 될 수도 있다

#열보다 행이 더 많은 test를 살펴보자.

In [ ]:
test.info()
# 타겟이 없기 때문에 열은 하나가 적음 (train int130개 test int129개)

#정수 열
#정수 열에서 고유 한 값의 분포를 살펴보고
#각 열에 대해 고유 값의 수를 세고 결과를 막대 그래프로 확인

In [ ]:
train.select_dtypes(np.int64).nunique().value_counts().sort_index().plot.bar(color = 'blue',
                                                                                 figsize = (8, 6),                                                                          
                                                                                 edgecolor = 'k', linewidth = 2);

plt.xlabel('Number of Unique Values');plt.ylabel('Count');
plt.title('Count of Unique Values in Integer Columns');

#정수열의 고유한 값의 분포를 살펴본 결과

In [ ]:
from collections import OrderedDict

plt.figure(figsize = (20, 16))
plt.style.use('fivethirtyeight')

#Color mapping
colors = OrderedDict({1: 'red', 2:'orange', 3:'blue', 4:'green'})
poverty_mapping = OrderedDict({1:'extreme', 2:'moderate', 3:'vulnerable', 4:'non vulnerable'})

#Interate through the float columns
for i, col in enumerate(train.select_dtypes('float')):
    ax = plt.subplot(4, 2, i+1)
    #Interate throught the poverty levels
    for poverty_level, color in colors.items():
        #plot each poverty level as a separate line
        sns.kdeplot(train.loc[train['Target'] == poverty_level, col].dropna(),
                   ax = ax,color = color, label = poverty_mapping[poverty_level])
        
        plt.title(f'{col.capitalize()} Distribution'); plt.xlabel(f'{col}');plt.ylabel('Desity')
        
        plt.subplots_adjust(top = 2)
        
        #v2al = mothly rent payment 월세 지불
        #b18q1 = number of tablets household owns 세대가 소유한 태블릿 수
        #rez_esc, Years behind in school 학교 몇년뒤에 가냐
        
        #overcrowding = persons per room  방 당 사람
        #SQBovercrowding = overcrowding squared 
      
        
        #dependency = dependency, Dependency rate, calculated = (number of members of the household younger than 19 or older than 64)/
                                                #(number of member of household between 19 and 64)
            
            #의존성, 의존성 비율, 계산 된 = (19 세 이하 또는 64 세 이상 가구 구성원 수) / (19-64 세 가구 구성원 수)
        #SQBdependency = dependency squared
        #SQBmeaned = square of the mean years of education of adults (>=18) in the household
        


위의 도표를 통해 모델과 가장 관련이있는 변수를 파악 할 수 있다.
그래프 중 하나를 설명하자면, 
성인평균 교육수준이 높을 수록 빈곤이 덜하다는 것을 알 수 있다

또한 피처 간의 관계를 측정하기 위해 변수와 대상 간의 상관 관계를 계산할 것 이다.

Object Columns

In [ ]:
train.select_dtypes('object').head()

In [ ]:
#dependency 오류남 왜인지는 모름

mapping = {"yes" : 1, "no" : 0}

#Apply the same operation to both train and test
for df in [train, test]:
    #Fill in the values with the correct mapping
    df['dependency'] = df['dependency'].replace(mapping).astype(np.float64)
    df['edjefa'] = df['edjefa'].replace(mapping).astype(np.float64)
    df['edjefe'] = df['edjefe'].replace(mapping).astype(np.float64)
    
    train[['dependency', 'edjefa', 'edjefe']].describe()
    

In [ ]:
plt.figure(figsize = (16, 12))

# Iterate through the float columns
for i, col in enumerate(['dependency', 'edjefa', 'edjefe']):
    ax = plt.subplot(3, 1, i + 1)
    # Iterate through the poverty levels
    for poverty_level, color in colors.items():
        # Plot each poverty level as a separate line
        sns.kdeplot(train.loc[train['Target'] == poverty_level, col].dropna(), 
                    ax = ax, color = color, label = poverty_mapping[poverty_level])
        
    plt.title(f'{col.capitalize()} Distribution'); plt.xlabel(f'{col}'); plt.ylabel('Density')

plt.subplots_adjust(top = 2)

#dependency, Dependency rate, calculated = (number of members of the household younger than 19 or older than 64)/(number of member of household between 19 and 64)
#edjefe, years of education of male head of household,based on the interaction of escolari (years of education), head of household and gender, yes=1 and no=0
#edjefa, years of education of female head of household, based on the interaction of escolari (years of education), head of household and gender, yes=1 and no=0


#이런 단계를 거치면 변수는 이제 숫자로 올바르게 표시되며 기계 학습 모델에 제공 될 수 있음

#위와 같은 작업을 좀 더 쉽게하기 위해 교육 및 테스트 데이터 프레임을 결합한다.
#기능 엔지니어링을 시작한 후에는 두 데이터 프레임에 동일한 작업을 적용하여
#동일한 기능을 사용하기 때문에 중요하다. 나중에 Target을 기준으로 세트를 분리 할 수 있다.

In [ ]:
# Add null Target column to test
test['Target'] = np.nan
data = train.append(test, ignore_index = True)


> 라벨 배포 탐색

라벨의 분포를 보면 문제가 얼마나 불균형한지 알 수 있음
4 가지 가능한 정수 수준이 있는데, 이는 4 가지 빈곤 수준을 나타냅니다.
(extreme, moderate vulnerable non vulnerable)

올바른 레이블을 확인하기 위해 parentesco1 == 1 인 열만 하위 세트합니다.
각 세대의 올바른 라벨 인 세대주이기 때문입니다. parentesco1, = 1 if 세대주

아래 막대 그림은  test 레이블이 없기 때문에 교육 레이블의 분포를 보여준다.

In [ ]:
# Heads of household
heads = data.loc[data['parentesco1'] == 1].copy()

# Labels for training
train_labels = data.loc[(data['Target'].notnull()) & (data['parentesco1'] == 1), ['Target', 'idhogar']]

# Value counts of target
label_counts = train_labels['Target'].value_counts().sort_index()

# Bar plot of occurrences of each label
label_counts.plot.bar(figsize = (8, 6), 
                      color = colors.values(),
                      edgecolor = 'k', linewidth = 2)

# Formatting
plt.xlabel('Poverty Level'); plt.ylabel('Count'); 
plt.xticks([x - 1 for x in poverty_mapping.keys()], 
           list(poverty_mapping.values()), rotation = 60)
plt.title('Poverty Level Breakdown');

label_counts

지금 우리는 불균형 class문제를 다루고 정리하는 중임

이때까지 정리하여 바차트로 확인해본 결과
non vulnerable이 가장 많고 extreme이 가장 낮은 것으로 나온다.

우리가 관심있는것은 빈곤층!


불균형 분류 문제의 한 가지 문제점은 머신 러닝 모델이 소수 클래스 예측도가 낮다.
우리가 빈곤을 분류하였는데 extreme보다 non vulnerable한 경우를 더 많이 본다면,
노출이 적기 때문에 빈곤 가정을 파악하기가 더 어려워진다.

클래스 불균형을 해결하는 한 가지 가능한 방법은 오버 샘플링을 이용하는 것.

잘못된 레이블 해결

실제 데이터 세트와 마찬가지로 Costa Rican Poverty 데이터에는 몇 가지 문제가 있다.

일반적으로 데이터 과학 프로젝트의 80 %는 cleaning data and fixing erros 하는데 사용한다.
사람 입력 오류, 측정 오류 또는 때로는 정확하지만 눈에 띄는 극단적 인 값일 수 있습니다.


이 데이터 문제의 경우, 같은 가구에 속한 개인의 빈곤 수준이 다르기 때문에 일부 레이블이 올바르지 않다.

이것이 왜 그런지에 대해서는 알려지지 않았지만 데이터제공처(The organizers)에서 세대주를
true label로 하는것을 추천했다.

이러한 정보를 통해 업무를 훨씬 쉽게 수행 할 수 있지만
실제 문제에서는 레이블이 왜 틀린지, 문제를 직접 해결하는 방법을 찾아야한다.

이 섹션에서는 레이블이 반드시 필요한 것은 아니지만 레이블 관련 문제를 해결할것이다.


-오류 식별
먼저 오류를 찾아 수정해야한다.
가족 구성원에 대해 다른 레이블이있는 세대를 찾으려면 세대별로 데이터를 그룹화 한 다음
대상의 고유 한 값이 하나만 있는지 확인해야한다.


In [ ]:
# Groupby the household and figure out the number of unique values
all_equal = train.groupby('idhogar')['Target'].apply(lambda x: x.nunique() == 1)

# Households where targets are not all equal
not_equal = all_equal[all_equal != True]
print('There are {} households where the family members do not all have the same target.'.format(len(not_equal)))

가족구성원중 같지않은 타겟을 가지고 있는 세대주가 85개 있다.

예시.

idhogar = 세대 수준 식별자
parentesco1, = 1 if 세대주

In [ ]:
train[train['idhogar'] == not_equal.index[0]][['idhogar', 'parentesco1', 'Target']]

The organizers는 올바른 레이블이 parentesco1 == 1 인 세대주라고 한다.

이 세대의 모든 구성원에 대해 올바른 레이블은 3입니다.
우리는 이 가구의 모든 개인을 올바른 빈곤 수준으로 재 할당함으로써
(나중에 보여지는 바와 같이)이를 수정한다.

또한 세대주가없는 가정은 동일한 세대의 개인에게
세대주 레이블을 할당하여 모든 상표 불일치를 수정할 계획 

In [ ]:
households_leader = train.groupby('idhogar')['parentesco1'].sum()

# Find households without a head
households_no_head = train.loc[train['idhogar'].isin(households_leader[households_leader == 0].index), :]

print('There are {} households without a head.'.format(households_no_head['idhogar'].nunique()))

In [ ]:
# Find households without a head and where labels are different
#head가 없는 세대주를 찾고 어느 라벨이 다른지 찾기
households_no_head_equal = households_no_head.groupby('idhogar')['Target'].apply(lambda x: x.nunique() == 1)
print('{} Households with no head have different labels.'.format(sum(households_no_head_equal == False)))

> 0 Households with no head have different labels.

head가없고 구성원의 레이블 값이 다른 세대주에 대해 걱정할 필요가 없음을 의미함.
이 문제에 대해 the organizers에 따르면 세대에 head가 없으면 true label이 없다.
따라서 head가 없는 세대주는 트레이닝 하지 않는다.

Correct Errors
이제, head가있는 가구와 빈곤층이 다른 세대에 대한 레이블을 수정한다.


In [ ]:
# Iterate through each household
for household in not_equal.index:
    # Find the correct label (for the head of household)
    true_target = int(train[(train['idhogar'] == household) & (train['parentesco1'] == 1.0)]['Target'])
    
    # Set the correct label for all members in the household
    train.loc[train['idhogar'] == household, 'Target'] = true_target
    
    
# Groupby the household and figure out the number of unique values
all_equal = train.groupby('idhogar')['Target'].apply(lambda x: x.nunique() == 1)

# Households where targets are not all equal
not_equal = all_equal[all_equal != True]
print('There are {} households where the family members do not all have the same target.'.format(len(not_equal)))

레이블에는 세대주만 사용하기 때문에 이 단계는 완전히 필요한 것은 아니지만
실제 상황에서 발생할 수있는 데이터 오류를 수정하기위한 workflow를 보여주는 것이다.

경력을 위해 연습하기 !

누락 된 변수
탐색적 데이터 분석의 가장 중요한 단계 중 하나는 데이터에서 누락 된 값을 찾아서 처리 방법을 결정하는 것.
기계 학습 모델을 사용하기 전에 누락 된 값을 채워야하며,
기능을 기반으로 값을 채우는 가장 좋은 전략을 고려해야한다.

먼저 각 열에서 결 측값의 백분율을 확인한다.

In [ ]:
# Number of missing in each column
missing = pd.DataFrame(data.isnull().sum()).rename(columns = {0: 'total'})

# Create a percentage missing
missing['percent'] = missing['total'] / len(data)

missing.sort_values('percent', ascending = False).head(10).drop('Target')

결측값이 높은 다른 3 개의 열을 처리해야함

v18q1 : 정제 수

<밑에는 뭔말인지 모름용>
가족이 소유 한 태블릿 수를 나타내는 v18q1부터 시작해보면.이 변수의 값 카운트를 볼 수 있다.
이것은 가계 변수이기 때문에 가계 수준에서만 살펴 보는 것이 합리적이므로 세대주를위한 행만 선택합니다.

값 카운트를 플롯하는 기능
다른 열에 대한 값 수를 플로팅하고 싶을 수도 있으므로 간단한 함수를 작성하면됩니다.

In [ ]:
def plot_value_counts(df, col, heads_only = False):
    """Plot value counts of a column, optionally with only the heads of a household"""
    # Select heads of household
    if heads_only:
        df = df.loc[df['parentesco1'] == 1].copy()
        
    plt.figure(figsize = (8, 6))
    df[col].value_counts().sort_index().plot.bar(color = 'blue',
                                                 edgecolor = 'k',
                                                 linewidth = 2)
    plt.xlabel(f'{col}'); plt.title(f'{col} Value Counts'); plt.ylabel('Count')
    plt.show();


"" "선택적으로 세대주 만 포함하는 열의 값 개수를 표시." ""

In [ ]:
plot_value_counts(heads, 'v18q1')


존재하는 데이터 만 사용하는 경우 가장 일반적인 태블릿 수는 1

v18q의 값을 그룹화하여 (태블릿의 경우 1, 그렇지 않은 경우 0) v18q1의 null 값 수를 계산할 수 있다.

v18q = 가족이 태블릿을 소유하고 있는지 여부
이 칼럼을 여러 태블릿과 결합하여 가설이 유지되는지 확인해야함.

(null 값이 가족이 태블릿을 소유하지 않음을 나타내는 지 알려준다.)

In [ ]:
heads.groupby('v18q')['v18q1'].apply(lambda x: x.isnull().sum())

v18q1에 대해 nan이있는 모든 가족은 태블릿을 소유하지 않는 것을 확인됨
따라서 이 결 측값을 0으로 채울 수 있다.

In [ ]:
data['v18q1'] = data['v18q1'].fillna(0)

In [ ]:
# Variables indicating home ownership
own_variables = [x for x in data if x.startswith('tipo')]


# Plot of the home ownership variables for home missing rent payments
data.loc[data['v2a1'].isnull(), own_variables].sum().plot.bar(figsize = (10, 8),
                                                                        color = 'green',
                                                              edgecolor = 'k', linewidth = 2);
plt.xticks([0, 1, 2, 3, 4],
           ['Owns and Paid Off', 'Owns and Paying', 'Rented', 'Precarious', 'Other'],
          rotation = 60)
plt.title('Home Ownership Status for Households Missing Rent Payments', size = 18);

V2a1 : 월 임대료 지불
다음으로 누락 된 열은 월 임대료 지불을 나타내는 v2a1

이 바차트는 월 임대료 지불에 대해 주택의 소유권 상태를 보여줍니다.



주택 소유 변수의 의미는 다음과 같다.

tipovivi1, = 1 소유 및 완전 지불 주택
tipovivi2, "= 1 소유, 할부 지불"
tipovivi3, = 1 임대
tipovivi4, = 1 불안정한 상태
tipovivi5, "= 1 기타 (지정, 차용)"

주로 : 월세를 지불하지 않는 가구는 일반적으로 자신의 집을 소유한다. 

소유하고 월 임대료가 누락 된 주택의 경우 임대료 지불액을 0으로 설정할 수 있다.
다른 주택의 경우 결측값을 대치 할 수 있지만 해당 가정에 결측값이 있음을 나타내는 플래그(부울)열을 추가한다.

In [ ]:
# Fill in households that own the house with 0 rent payment
data.loc[(data['tipovivi1'] == 1), 'v2a1'] = 0

# Create missing rent payment column
data['v2a1-missing'] = data['v2a1'].isnull()

data['v2a1-missing'].value_counts()

rez_esc 

결측값 비율이 높은 마지막 열은 rez_esc이며 학교에서 몇 년이 지났음을 나타낸다.
null 값을 가진 가정의 경우 현재 학교에 자녀가 없을 가능성이 있기때문에 이 열에서 결측값이없는 사람의 연령과
결측값이없는 사람의 연령을 찾아서 이것을 테스트 해본다.

In [ ]:
data.loc[data['rez_esc'].notnull()]['age'].describe()

가장 가치가없는 나이: 17 세
이보다 나이가 많은 사람은 아마도 학교에 있지 않다고 가정 할 수 도 있다.

이 변수는 7과 19 사이의 개인에 대해서만 정의
이 범위보다 젊거나 더 오래된 사람은 아마도 몇 년 뒤 학교에 없기 때문에 그 값은 0으로 설정해야함.
이 변수는 개인이 19 세 이상이고 결측값이 있거나 7보다 작고 결측값이있는 경우 0으로 설정할 수 있습니다.
다른 사람에게는 값을 대치하고 부울 플래그를 추가합니다.

In [ ]:
# If individual is over 19 or younger than 7 and missing years behind, set it to 0
data.loc[((data['age'] > 19) | (data['age'] < 7)) & (data['rez_esc'].isnull()), 'rez_esc'] = 0

# Add a flag for those between 7 and 19 with a missing value
data['rez_esc-missing'] = data['rez_esc'].isnull()

rez_esc 열에는 이상치가 하나있다.
변수의 최대 값이 5임. 따라서 5보다 큰 값은 5로 설정해야한다.

In [ ]:
data.loc[data['rez_esc'] > 5, 'rez_esc'] = 5

 점의 크기가 각 x- 값으로 표시되는 주어진 y- 값의 백분율을 나타내는 두 범주 형의 산점도.

In [ ]:
def plot_categoricals(x, y, data, annotate = True):
    """Plot counts of two categoricals.
    Size is raw count for each grouping.
    Percentages are for a given value of y."""
    
    # Raw counts 
    raw_counts = pd.DataFrame(data.groupby(y)[x].value_counts(normalize = False))
    raw_counts = raw_counts.rename(columns = {x: 'raw_count'})
    
    # Calculate counts for each group of x and y
    counts = pd.DataFrame(data.groupby(y)[x].value_counts(normalize = True))
    
    # Rename the column and reset the index
    counts = counts.rename(columns = {x: 'normalized_count'}).reset_index()
    counts['percent'] = 100 * counts['normalized_count']
    
    # Add the raw count
    counts['raw_count'] = list(raw_counts['raw_count'])
    
    plt.figure(figsize = (14, 10))
    # Scatter plot sized by percent
    plt.scatter(counts[x], counts[y], edgecolor = 'k', color = 'lightgreen',
                s = 100 * np.sqrt(counts['raw_count']), marker = 'o',
                alpha = 0.6, linewidth = 1.5)
    
    if annotate:
        # Annotate the plot with text
        for i, row in counts.iterrows():
            # Put text with appropriate offsets
            plt.annotate(xy = (row[x] - (1 / counts[x].nunique()), 
                               row[y] - (0.15 / counts[y].nunique())),
                         color = 'navy',
                         s = f"{round(row['percent'], 1)}%")
        
    # Set tick marks
    plt.yticks(counts[y].unique())
    plt.xticks(counts[x].unique())
    
    # Transform min and max to evenly space in square root domain
    sqr_min = int(np.sqrt(raw_counts['raw_count'].min()))
    sqr_max = int(np.sqrt(raw_counts['raw_count'].max()))
    
    # 5 sizes for legend
    msizes = list(range(sqr_min, sqr_max,
                        int(( sqr_max - sqr_min) / 5)))
    markers = []
    
    # Markers for legend
    for size in msizes:
        markers.append(plt.scatter([], [], s = 100 * size, 
                                   label = f'{int(round(np.square(size) / 100) * 100)}', 
                                   color = 'lightgreen',
                                   alpha = 0.6, edgecolor = 'k', linewidth = 1.5))
        
    # Legend and formatting
    plt.legend(handles = markers, title = 'Counts',
               labelspacing = 3, handletextpad = 2,
               fontsize = 16,
               loc = (1.10, 0.19))
    
    plt.annotate(f'* Size represents raw count while % is for a given y value.',
                 xy = (0, 1), xycoords = 'figure points', size = 10)
    
    # Adjust axes limits
    plt.xlim((counts[x].min() - (6 / counts[x].nunique()), 
              counts[x].max() + (6 / counts[x].nunique())))
    plt.ylim((counts[y].min() - (4 / counts[y].nunique()), 
              counts[y].max() + (4 / counts[y].nunique())))
    plt.grid(None)
    plt.xlabel(f"{x}"); plt.ylabel(f"{y}"); plt.title(f"{y} vs {x}");

In [ ]:
plot_categoricals('rez_esc', 'Target', data);

마커의 크기는 원시 수를 나타낸다.
그림을 읽으려면 주어진 y 값을 선택한 다음 행을 읽는다.

예를 들어, 빈곤 수준이 1 인 경우, 개인의 93 %가 약 800 명으로 총 수에 뒤쳐지지 않으며,
약 0.4 %의 개인이 5 년 뒤에이 범주에서 약 50 명의 전체 개인을 갖는다.
이 플롯은 전체 개수와 범주 내 비율을 모두 표시하려고 시도합니다. 

In [ ]:
plot_categoricals('escolari', 'Target', data, annotate = False)

In [ ]:
plot_value_counts(data[(data['rez_esc-missing'] == 1)], 
                  'Target')


여기서 분포는 모든 데이터의 분포와 일치하는 것으로 보인다.

(poverty level breakdown 그래프 (빨노파초))

In [ ]:
plot_value_counts(data[(data['v2a1-missing'] == 1)], 
                  'Target')

<뭔말인지 이해하지 못함>
이것은 빈곤율이 보통 인 빈곤율이 높을수록 빈곤의 지표가 될 수있는 것처럼 보인다.

이것은 중요한 요점을 나타냅니다. 때로는 누락 된 정보가 귀하가 제공 한 정보만큼 중요합니다.

*뛰어넘기*

Model Selection

In [ ]:
# Model imports
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegressionCV, RidgeClassifierCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

Model Optimization


모델 최적화는 교차 검증을 통해 하이퍼 파라미터를 조정하여 기계 학습 모델에서 최상의 성능을 추출하는 프로세스이다.
최상의 모델 하이퍼 파라미터는 모든 데이터 세트마다 다르기 때문에이 작업이 필요하다.


1.설명서
2.그리드 검색
3.무작위 검색
4.자동화 된 최적화

이 방법은 일반적으로 가장 효율적인 방법이며 Tree Parzen Estimator와 함께 수정 된 Bayesian Optimization 버전을 사용하는 Hyperopt를 포함한 여러 라이브러리에서 쉽게 구현할 수 있기 때문에 4가지를 고수 할 것이다.

Hyperopt를 사용한 모델 튜닝
베이지안 최적화에는 4 가지 부분이 필요함.

목표 함수 : 우리가 최대화하고 싶은 것
도메인 공간 : 검색 할 지역
다음 하이퍼 파라미터 선택 알고리즘 : 과거 결과를 사용하여 다음 값 제안
결과 기록 : 과거 결과를 저장
이전에 Hyperopt 사용에 대해 작성 했으므로 여기서는 구현을 한다.

In [ ]:
from hyperopt import hp, tpe, Trials, fmin, STATUS_OK
from hyperopt.pyll.stochastic import sample
import csv
import ast
from timeit import default_timer as timer

1. 목적 함수
모델 하이퍼 파라미터를 가져와 관련 유효성 검사 점수를 반환합니다 Hyperopt는 최소화하기 위해 점수가 필요하므로 1-Macro F1 점수를 반환합니다.

In [ ]:
def objective(hyperparameters, nfolds=5):
    """Return validation score from hyperparameters for LightGBM"""
    
    # Keep track of evals
    global ITERATION
    ITERATION += 1
    
    # Retrieve the subsample
    subsample = hyperparameters['boosting_type'].get('subsample', 1.0)
    subsample_freq = hyperparameters['boosting_type'].get('subsample_freq', 0)
    
    boosting_type = hyperparameters['boosting_type']['boosting_type']
    
    if boosting_type == 'dart':
        hyperparameters['drop_rate'] = hyperparameters['boosting_type']['drop_rate']
    
    # Subsample and subsample frequency to top level keys
    hyperparameters['subsample'] = subsample
    hyperparameters['subsample_freq'] = subsample_freq
    hyperparameters['boosting_type'] = boosting_type
    
    # Whether or not to use limit maximum depth
    if not hyperparameters['limit_max_depth']:
        hyperparameters['max_depth'] = -1
    
    # Make sure parameters that need to be integers are integers
    for parameter_name in ['max_depth', 'num_leaves', 'subsample_for_bin', 
                           'min_child_samples', 'subsample_freq']:
        hyperparameters[parameter_name] = int(hyperparameters[parameter_name])

    if 'n_estimators' in hyperparameters:
        del hyperparameters['n_estimators']
    
    # Using stratified kfold cross validation
    strkfold = StratifiedKFold(n_splits = nfolds, shuffle = True)
    
    # Convert to arrays for indexing
    features = np.array(train_selected)
    labels = np.array(train_labels).reshape((-1 ))
    
    valid_scores = []
    best_estimators = []
    run_times = []
    
    model = lgb.LGBMClassifier(**hyperparameters, class_weight = 'balanced',
                               n_jobs=-1, metric = 'None',
                               n_estimators=10000)
    
    # Iterate through the folds
    for i, (train_indices, valid_indices) in enumerate(strkfold.split(features, labels)):
        
        # Training and validation data
        X_train = features[train_indices]
        X_valid = features[valid_indices]
        y_train = labels[train_indices]
        y_valid = labels[valid_indices]
        
        start = timer()
        # Train with early stopping
        model.fit(X_train, y_train, early_stopping_rounds = 100, 
                  eval_metric = macro_f1_score, 
                  eval_set = [(X_train, y_train), (X_valid, y_valid)],
                  eval_names = ['train', 'valid'],
                  verbose = 400)
        end = timer()
        # Record the validation fold score
        valid_scores.append(model.best_score_['valid']['macro_f1'])
        best_estimators.append(model.best_iteration_)
        
        run_times.append(end - start)
    
    score = np.mean(valid_scores)
    score_std = np.std(valid_scores)
    loss = 1 - score
    
    run_time = np.mean(run_times)
    run_time_std = np.std(run_times)
    
    estimators = int(np.mean(best_estimators))
    hyperparameters['n_estimators'] = estimators
    
    # Write to the csv file ('a' means append)
    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, hyperparameters, ITERATION, run_time, score, score_std])
    of_connection.close()
    
    # Display progress
    if ITERATION % PROGRESS == 0:
        display(f'Iteration: {ITERATION}, Current Score: {round(score, 4)}.')
    
    return {'loss': loss, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'time': run_time, 'time_std': run_time_std, 'status': STATUS_OK, 
            'score': score, 'score_std': score_std}

2. 검색 공간
도메인은 검색하려는 전체 값 범위이다.
유일하게 어려운 부분은 boosting_type = "goss"인 경우 1.0으로 설정해야하는 서브 샘플 비율이다.

In [ ]:
# Define the search space
space = {
    'boosting_type': hp.choice('boosting_type', 
                              [{'boosting_type': 'gbdt', 
                                'subsample': hp.uniform('gdbt_subsample', 0.5, 1),
                                'subsample_freq': hp.quniform('gbdt_subsample_freq', 1, 10, 1)}, 
                               {'boosting_type': 'dart', 
                                 'subsample': hp.uniform('dart_subsample', 0.5, 1),
                                 'subsample_freq': hp.quniform('dart_subsample_freq', 1, 10, 1),
                                 'drop_rate': hp.uniform('dart_drop_rate', 0.1, 0.5)},
                                {'boosting_type': 'goss',
                                 'subsample': 1.0,
                                 'subsample_freq': 0}]),
    'limit_max_depth': hp.choice('limit_max_depth', [True, False]),
    'max_depth': hp.quniform('max_depth', 1, 40, 1),
    'num_leaves': hp.quniform('num_leaves', 3, 50, 1),
    'learning_rate': hp.loguniform('learning_rate', 
                                   np.log(0.025), 
                                   np.log(0.25)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 2000, 100000, 2000),
    'min_child_samples': hp.quniform('min_child_samples', 5, 80, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.5, 1.0)
}


In [ ]:
sample(space)

3. 알고리즘
다음 값을 선택하는 알고리즘은 목적 함수의 대리 모델을 구성하기 위해 베이즈 규칙을 사용하는 Tree Parzen Estimator이다.
알고리즘은 목적 함수를 최대화하는 대신 대리 모델의 EI (Expected Improvement)를 최대화이다.

In [ ]:
algo = tpe.suggest

4. 결과 기록
결과를 기록하기 위해 두 가지 방법을 사용

평가판 개체 : 목적 함수에서 반환 된 모든 내용을 저장한다.
반복 할 때마다 CSV 파일에 쓰기
중복을 의미하기 때문에 진행 상황을 추적하기 위해 여러 가지 방법을 사용하는 것이 좋다.
한 가지 방법은 실패 할 수 있지만 두 가지 방법 모두 그렇지 않습니다! csv 파일을 사용하여 메서드가 실행되는 동안 메서드를 모니터링하고 Trials 개체를 저장 한 다음 다시로드하여 최적화를 다시 시작할 수 있다.

In [ ]:
#여기부터 내가 뛰어넘어서 오류 및 제대로 결과값이 나오지 않음. 본문 보면서 설명하기
# Record results
trials = Trials()

# Create a file and open a connection
OUT_FILE = 'optimization.csv'
of_connection = open(OUT_FILE, 'w')
writer = csv.writer(of_connection)

MAX_EVALS = 100
PROGRESS = 10
N_FOLDS = 5
ITERATION = 0

# Write column names
headers = ['loss', 'hyperparameters', 'iteration', 'runtime', 'score', 'std']
writer.writerow(headers)
of_connection.close()

In [ ]:
%%capture --no-display
display("Running Optimization for {} Trials.".format(MAX_EVALS))

# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest, trials = trials,
            max_evals = MAX_EVALS)

o 훈련을 다시 시작하면 동일한 평가판 개체를 전달하고 최대 반복 횟수를 늘릴 수 있다.
나중에 사용하기 위해 평가판을 json으로 저장할 수 있다.

In [ ]:
import json

# Save the trial results
with open('trials.json', 'w') as f:
    f.write(json.dumps(str(trials)))

Using Optimized Model

In [ ]:
results = pd.read_csv(OUT_FILE).sort_values('loss', ascending = True).reset_index()
results.head()

In [ ]:
plt.figure(figsize = (8, 6))
sns.regplot('iteration', 'score', data = results);
plt.title("Optimization Scores");
plt.xticks(list(range(1, results['iteration'].max() + 1, 3)));
